In [ ]:
!pip install tensorflowjs

    100% |████████████████████████████████| 84.7MB 467kB/s 
    100% |████████████████████████████████| 13.9MB 3.5MB/s 
    100% |████████████████████████████████| 358kB 19.9MB/s 
    100% |████████████████████████████████| 61kB 23.3MB/s 
    100% |████████████████████████████████| 3.0MB 11.9MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


### TODO:
__need__:
* balance classes ?

__would be good__

__experiment__

* нейронки люблять нормалізовані центровані дані
* збільшуєш к-ть нйронів, потім зменшуєш
* на 5-6 леєрі градієнт затухає, в денс мережі, якшо вона без накруток якихось
* якшо кост стрибає - зм learning rate(hard in keras, там decay)
 
__questions__

* як ставити дропаути
* evernote, reader view

### Read data

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
features = np.genfromtxt('train_test_data/features', delimiter=',')
vectors_test = np.genfromtxt('train_test_data/vectors_test', delimiter=',')
bin_tes_y = np.genfromtxt('train_test_data/bin_tes_y', delimiter=',')
bin_tr_y = np.genfromtxt('train_test_data/bin_tr_y', delimiter=',')

# y_test = np.genfromtxt('train_test_data/y_test', delimiter=',')
# y_train = np.genfromtxt('train_test_data/y_train', delimiter=',')
# x_train = np.genfromtxt('train_test_data/x_train', delimiter=',')
# x_test = np.genfromtxt('train_test_data/x_test', delimiter=',')

### Model & train

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
simple_model = Sequential([Dense(200,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(150,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.4, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(50,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.2, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])
ES_callback = EarlyStopping(monitor='val_binary_accuracy', min_delta=0.00005, patience=4)

filepath="improvement-{epoch:02d}-{loss:.4f}-{val_binary_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_binary_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['binary_accuracy'])

In [ ]:
simple_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 150)               10601850  
_________________________________________________________________
dropout_5 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 200)               30200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 150)               30150     
_________________________________________________________________
dropout_7 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                7550      
__________

In [ ]:
model_output= simple_model.fit(features,
                               bin_tr_y,
                               batch_size=128,
                               epochs=40,
                               verbose=1,
                               validation_data=(vectors_test, bin_tes_y),
                               callbacks=[ES_callback, checkpoint])

In [ ]:
simple_model = Sequential([Dense(150,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)),
                           Dropout(0.6, noise_shape=None, seed=None),
                           Dense(200,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.4, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

simple_model.load_weights('improvement-10-0.9714-0.9181.hdf5')

simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['binary_accuracy'])

### Model evaluation


In [ ]:
predicted = simple_model.predict_classes(vectors_test)

In [ ]:
confusion_matrix(bin_tes_y, predicted)

array([[11245,  1409],
       [  877, 14390]])

In [ ]:
print(classification_report(bin_tes_y, predicted))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91     12654
           1       0.91      0.94      0.93     15267

   micro avg       0.92      0.92      0.92     27921
   macro avg       0.92      0.92      0.92     27921
weighted avg       0.92      0.92      0.92     27921



### Grid search

parameters:

__vectorizer__
* min_df 0.0001 4
* max_df 0.15 ...

__model__
* layers
* batch_size    512, 256
* epochs        20, 40, 
* dense 100-300
* dropout

In [1]:
# cut data
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(x_test, y_test, stratify=y_test)
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(la_x_test, la_y_test, stratify=la_y_test)
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(la_x_train, la_y_train, stratify=la_y_train)
la_x_train, la_x_test, la_y_train, la_y_test = train_test_split(la_x_train, la_y_train, stratify=la_y_train)
gr_x_train, gr_x_test, gr_y_train, gr_y_test = train_test_split(la_x_train, la_y_train, stratify=la_y_train)

NameError: name 'train_test_split' is not defined

In [ ]:
gr_y_train = np.isin(gr_y_train.values, tech).astype(np.uint8)
gr_y_test = np.isin(gr_y_test.values, tech).astype(np.uint8)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

ES_callback = EarlyStopping(monitor='loss', min_delta=0.00005, patience=4)

def create_model(neurons=100,neurons2=100, dropout=0.5,dropout2=0.5, layers=1):
    
    model = Sequential()
    model.add(Dense(neurons,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)))
    model.add(Dropout(dropout, noise_shape=None, seed=None))
    if layers>=2:                   
        model.add(Dense(neurons2,activation='relu', kernel_regularizer=l2(0.01)))
        model.add(Dropout(dropout2, noise_shape=None, seed=None))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=1)

In [ ]:
# define the grid search parameters
neurons = [100, 150]
layers = [1,2]
batch_size = [512]
dropout = [0.6]
# batch_size = [128, 256, 512]
# dropout = [0.35, 0.45, 0.6]

callbacks = [[ES_callback]]
validation_data=[(tfidf.transform(gr_x_test), gr_y_test)]
epochs = [10]

param_grid = dict(neurons=neurons,
                  neurons2=neurons,
                  dropout=dropout,
                  dropout2=dropout, 
                  layers=layers,
                 batch_size=batch_size,
                 callbacks=callbacks,
                  validation_data=validation_data,
                 epochs=epochs)

grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
#                     n_jobs=-1,
                   verbose=1)

grid_result = grid.fit(tfidf.transform(gr_x_train), gr_y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.763134 using {'batch_size': 512, 'callbacks': [<keras.callbacks.EarlyStopping object at 0x7f602a096f98>], 'dropout': 0.6, 'dropout2': 0.6, 'epochs': 10, 'layers': 2, 'neurons': 100, 'neurons2': 150, 'validation_data': (<736x92743 sparse matrix of type '<class 'numpy.float64'>'
	with 23265 stored elements in Compressed Sparse Row format>, array([0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1

### Saving the most successfully weighted model for js and for test

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(simple_model, 'model')